**MCQ Assessment: Python Fundamentals, NumPy, Pandas, and Matplotlib**
1. a, b, c
2. b
3. d
4. a
5. a, b
6. d
7. c
8. a
9. b
10. a, c
11. b, c
12. c
13. a, b, c
14. a, b, c
15. a
16. c
17. a
18. a
19. a
20. b
21. b
22. b
23. a
24. d
25. d
26. a
27. a
28. a
29. b
30. c


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
train_df = pd.read_csv(r"E:\\College\\LnB\\Take Away Assignment\\covid19-global-forecasting-week-5\\train.csv", low_memory= False)
test_df = pd.read_csv(r"E:\\College\\LnB\\Take Away Assignment\\covid19-global-forecasting-week-5\\test.csv")

In [ ]:
train_df

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df[train_df["Country_Region"] == "US"]

In [ ]:
train_df = train_df[(train_df["County"].isna())]

In [ ]:
train_df[train_df["Country_Region"] == "US"]

In [ ]:
train_df["Country_Region"].unique()

In [ ]:
country_df = train_df[(train_df['Country_Region'].notna()) & (train_df['Province_State'].notna())].iloc[:,[2,3]]
country_df

In [ ]:
country_df['Country_Region'].unique()

In [ ]:
train_df

In [ ]:
train_df['Date'] = pd.to_datetime(train_df['Date'])

In [ ]:
train_df.info()

In [ ]:
group_by_country = train_df.groupby(['Country_Region', 'Target'])['TargetValue'].sum().unstack()
group_by_country["ConfirmedCases"].idxmax()

In [ ]:
group_by_country["Fatalities"].idxmax()

In [ ]:
group_by_country["Recovered"] = (group_by_country["ConfirmedCases"]) - (group_by_country["Fatalities"])

In [ ]:
group_by_country

In [ ]:
group_by_country["Recovered"].idxmax()

In [ ]:
group_by_country

In [ ]:
train_df['day_of_week'] = train_df['Date'].dt.dayofweek
train_df['day_of_month'] = train_df['Date'].dt.day
train_df['week_of_year'] = train_df['Date'].dt.isocalendar().week
train_df['month'] = train_df['Date'].dt.month
train_df['quarter'] = train_df['Date'].dt.quarter
train_df['year'] = train_df['Date'].dt.year
train_df['is_weekend'] = train_df['Date'].dt.dayofweek >= 5   # is_weekend: Boolean indicating if the day is a weekend (True for Saturday and Sunday)
train_df['day_of_year'] = train_df['Date'].dt.dayofyear

In [ ]:
train_df

In [ ]:
newdf = train_df[["Country_Region", "County", "Province_State", "Population"]].drop_duplicates()

In [ ]:
newdf

In [ ]:
newdf = newdf.reset_index()

In [ ]:
newdf[newdf["Country_Region"].isin(['Australia', 'Canada', 'China', 'Denmark', 'France', 'Netherlands', 'US', 'United Kingdom'])]

In [ ]:
grp_population_1 = newdf.groupby(["Country_Region"])["Population"].sum().reset_index()
grp_population_1

In [ ]:
grp_population_1[grp_population_1["Country_Region"] == "Australia"]

In [ ]:
train_df[(train_df["Country_Region"] == "US") & (train_df["Province_State"].isna()) & (train_df["County"].isna())]

In [ ]:
grp_population_1[grp_population_1["Country_Region"] == "US"]

In [ ]:
grp_population = newdf.groupby(["Country_Region", "Province_State"])["Population"].sum().reset_index()

In [ ]:
grp_population[grp_population["Country_Region"] == "Austra"]

In [ ]:
grp_population[grp_population["Country_Region"] == "US"]

In [ ]:
grp_population[grp_population["Country_Region"] == "US"]["Population"].sum()

In [ ]:
grp_population[grp_population["Country_Region"] == "Australia"]["Population"].sum()

In [ ]:
# def population(df: pd.DataFrame, result: pd.DataFrame) -> pd.DataFrame:
#     for _, row in df.iterrows():
#         if pd.isna(row["County"]) and pd.isna(row["Province_State"]):
#             if row["Country_Region"] not in result["Country_Region"].values:
#                 result.loc[len(result)] = row["Country_Region"]
#             if pd.isna(result["Population"]):
#                 result.loc[result["Country_Region"] == row["Country_Region"],"Population"] = row["Country_Region"]
#         else:
#             if row["Country_Region"] not in result["Country_Region"].values:
#                 result.loc[len(result)] = row[["Country_Region", "Population"]]
#             if pd.notna(row["County"]) and row["County"] not in result["County"].values:
#                 result.loc[len(result)] = row[["Country_Region", "County", "Population"]]
#             if pd.notna(row["Province_State"]) and row["Province_State"] not in result["Province_State"].values:
#                 result.loc[len(result)] = row[["Country_Region", "Province_State", "Population"]]
#     return result

# a = population(train_df, newdf)

In [84]:
import pandas as pd
# !pip install yfinance
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from datetime import date

# Get user input for stock and time period
whichstock = input("Which stock? > ")
howmanyyears = int(input("How many years? > "))

# Download historical stock data
today = date.today()
END_DATE = today.isoformat()
START_DATE = date(today.year - howmanyyears, today.month, today.day).isoformat()
data = yf.download(whichstock, start=START_DATE, end=END_DATE)

# Reset index and convert to datetime
data.reset_index(inplace=True)
data['Date'] = pd.to_datetime(data.Date)
data['Date'] = data['Date'].apply(lambda x: x.timestamp())

# Calculate Exponential Moving Averages (EMAs)
data['EMA-50'] = data['Close'].ewm(span=50, adjust=False).mean()
data['EMA-200'] = data['Close'].ewm(span=200, adjust=False).mean()

# Create feature engineering
data['Volume_Ratio'] = data['Volume'] / data['Volume'].rolling(window=20).mean()
data['Price_Change'] = data['Close'].pct_change()

# Scale data using Min-Max Scaler
scaler = MinMaxScaler()
data[['Open', 'High', 'Low', 'Close', 'Volume', 'EMA-50', 'EMA-200', 'Volume_Ratio', 'Price_Change']] = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close', 'Volume', 'EMA-50', 'EMA-200', 'Volume_Ratio', 'Price_Change']])

# Split data into training and testing sets
X = data.drop(['Close', 'Date'], axis=1)
y = data['Close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Make predictions
y_pred = model.predict(X_test)

y_test = y_test.replace([np.inf, -np.inf], np.nan).dropna()

y_pred[np.isinf(y_pred)] = np.nan
y_pred = y_pred[~np.isnan(y_pred)]
y_pred = y_pred.reshape(-1,)
# Evaluate model using Mean Squared Error
mse = np.nanmean((y_test - y_pred) ** 2)
print(f'Mean Squared Error: {mse:.2f}')

# Plot predicted prices
plt.plot(y_test, label='Actual Prices')
plt.plot(y_pred, label='Predicted Prices')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Stock Price Prediction')
plt.legend()
plt.show()

# Use model to predict future prices
future_prices = []
for i in range(3):  # predict prices for the next 3 months
    future_date = data.index[-1] + pd.DateOffset(months=i+1)
    future_data = pd.DataFrame({'Open': [data['Open'].iloc[-1]], 
                                'High': [data['High'].iloc[-1]], 
                                'Low': [data['Low'].iloc[-1]], 
                                'Close': [data['Close'].iloc[-1]], 
                                'Volume': [data['Volume'].iloc[-1]], 
                                'EMA-50': [data['EMA-50'].iloc[-1]], 
                                'EMA-200': [data['EMA-200'].iloc[-1]], 
                                'Volume_Ratio': [data['Volume_Ratio'].iloc[-1]], 
                                'Price_Change': [data['Price_Change'].iloc[-1]]})
    future_data = scaler.transform(future_data)
    future_price = model.predict(future_data)[0]
    future_prices.append((future_date, future_price))

# Plot predicted future prices
plt.plot([x[0] for x in future_prices], [x[1] for x in future_prices], label='Predicted Future Prices')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Stock Price Prediction')
plt.legend()
plt.show()

[*********************100%***********************]  1 of 1 completed
Epoch 1/100
32/32 [==============================] - 4s 27ms/step - loss: nan - val_loss: nan
Epoch 2/100
32/32 [==============================] - 0s 7ms/step - loss: nan - val_loss: nan
Epoch 3/100
32/32 [==============================] - 0s 8ms/step - loss: nan - val_loss: nan
Epoch 4/100
32/32 [==============================] - 0s 8ms/step - loss: nan - val_loss: nan
Epoch 5/100
8/8 [==============================] - 1s 2ms/step


ValueError: operands could not be broadcast together with shapes (252,) (0,) 